In [ ]:
import spacy
import pandas as pd
import re

In [ ]:
ok = pd.read_csv(r"F:\Corpus\finaux\lemondediplomatique1.csv", sep='\t', encoding='utf8')

In [ ]:
ok = ok[:10]

In [ ]:
ok['name'] = ok['name'].apply(lambda x : re.sub('^20', '', x))

In [ ]:
ok

In [ ]:
def generate_corpus_index(corpus_name):
    # Convert to lowercase and keep only alphanumeric characters
    cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', corpus_name.lower())
    # Ensure length is up to 10 characters, pad with '0' if needed
    return cleaned_name[:10].ljust(10, '0')

def generate_sub_corpus_index(corpus_index, sub_corpus_name):
    # Convert sub-corpus name to alphanumeric and pad with '0'
    cleaned_sub_corpus_name = re.sub(r'[^a-zA-Z0-9]', '', sub_corpus_name)[:10].ljust(10, '0')
    return f"{corpus_index}_{cleaned_sub_corpus_name}"

def generate_turn_index(sub_corpus_index, turn_number):
    # The turn index is constructed by concatenating the sub-corpus index and the turn number,
    return f"{sub_corpus_index}_{str(turn_number)}"

def generate_speaker_index(sub_corpus_index, spk_name):
    # Convert speaker name to alphanumeric
    try : 
        cleaned_spk_name = re.sub(r'[^a-zA-Z0-9]', '', spk_name)
        # Combine the corpus index and sub-corpus index, and spk name with an underscore to form the speaker index
        speaker_index = f"{sub_corpus_index}_{cleaned_spk_name}"
    except : 
        speaker_index = ''
    # Return the generated speaker index
    return speaker_index

In [ ]:
ok

In [ ]:
# Load the language model
nlp = spacy.load('fr_core_news_sm')

# Your custom function with nlp.pipes
def blabla(nlp, texts):
    docs = list(nlp.pipe(texts, batch_size=128)) 
    return [list(doc.sents) for doc in docs]

In [ ]:
ok['texte'] = ok['text'].fillna('')
# Applying your custom function with nlp.pipes to get doc.sents for the 'texte' column
ok['doc_sents'] = blabla(nlp, ok['text'])

ok1 = ok.explode('doc_sents')
ok1 = ok1.reset_index(drop=True)

In [ ]:
def create_index(df, corpus_name):
    #vérification dans la base de donénes à faire
    df['index_corpus']=generate_corpus_index(corpus_name)
    df['index_sub_corpus'] = df.apply(lambda row : generate_sub_corpus_index(row['index_corpus'], row['name']), axis=1)
    df['nb_index_enregistrement'] = df['index_sub_corpus'].ne(df['index_sub_corpus'].shift()).cumsum()
    df['numero_tour'] = df.groupby('nb_index_enregistrement').cumcount() + 1
    df['index_turn_index'] = df.apply(lambda row : generate_turn_index(row['index_sub_corpus'], row['numero_tour']), axis=1)
    df['index_spk']= df.apply(lambda row : generate_sub_corpus_index(row['index_sub_corpus'], row['author']), axis=1)
    return df

In [ ]:
ok1['texte']= ok1['doc_sents'].apply(lambda x : x.text)

ok1 = ok1.fillna('')

ok1 = create_index(ok1, 'LEMONDEDIPLOMATIQUE')

ok1 = ok1.drop('text', axis=1)

In [ ]:
ok1

In [ ]:
def get_MD(list_):
    tokens = list(enumerate(list_))
    list_md = ['car','bref','comme','donc','enfin','ensuite','après','puis','puisque','soudain','mais', 'ben']
    index_tokens = [(ind,tok) for ind, tok in tokens if tok.text.lower() in list_md]
    return index_tokens

In [ ]:
ok1['new']=ok1['doc_sents'].apply(lambda x: get_MD(x))

In [ ]:
def keep_words(string):
    res = re.sub(r'[^\w\s]', '', string)
    string = string.replace(' ', '')
    return res

In [ ]:
def create_index(df):
    #vérification dans la base de donénes à faire
    df['index_corpus']=[keep_words(x.lower().replace(' ', '')) for x in df['journal'].tolist()]
    df['index_enregistrement']= [keep_words(x.lower()) +'_'+ keep_words(y.lower()) for x,y in list(zip(df['journal'].tolist(), df['titre'].tolist()))]
    df['index_locuteur']=[x.lower()+'_'+keep_words(y.lower().replace('_', '').replace('.', '')) for x,y in list(zip(df['index_enregistrement'].tolist(), df['auteur'].tolist())) ]
    df['nb_index_enregistrement'] = df['index_enregistrement'].ne(df['index_enregistrement'].shift()).cumsum()
    df['numero_tour'] = df.groupby('nb_index_enregistrement').cumcount() + 1
    df['index_tour'] = [x.lower()+'_'+ str(y) for x,y in list(zip(df['index_enregistrement'].tolist(), df['numero_tour'].tolist()))]
    return df

In [ ]:
ok1.columns

In [ ]:
ok2 = create_index(ok1.fillna(''))

In [ ]:
for ind, x in enumerate(ok2['doc_sents'].tolist()[0]):
    if x.text =='Faute':
        print(ind, x)

In [ ]:
def tokenize(x):
    list_md = ['car','bref','comme','donc','enfin','ensuite','après','puis','puisque','soudain','mais', 'ben']
    index_tokens = [(ind,tok) for ind, tok in tokens if tok in list_md]
    return index_tokens

In [ ]:
def create_texte(doc):
    if isinstance(doc, str):
        return ''
    else:
        return doc.text
    

In [ ]:
def tokenize(x):
    if isinstance(x, str):
        tokens= list(enumerate(word_tokenize(x, language='french')))
        list_md = ['car','bref','comme','donc','enfin','ensuite','après','puis','puisque','soudain','mais', 'ben']
        index_tokens = [(ind,tok) for ind, tok in tokens if tok in list_md]
        return index_tokens
    else:
        return []

In [ ]:
def create_tokens_index(tokens):
    list_md = ['ben', 'mais', 'enfin','oui']
    index_tokens = [(ind,tok.text) for ind, tok in enumerate(tokens) if tok.text.lower() in list_md]
    return index_tokens

In [ ]:
ok2['texte']= ok2['doc_sents'].apply(lambda x : create_texte(x))

In [ ]:
ok2['index_token'] = ok2['doc_sents'].apply(lambda x : create_tokens_index(x))

In [ ]:
for x in ok2['index_token'].tolist():
    if len(x)>0:
        pass
    else:
        print(x)
        print(type(x))

In [ ]:
ok['texte'] = ok['texte'].fillna('')
ok['doc_sents'] = blabla(nlp, ok['texte'])
ok = ok.explode('doc_sents')

In [ ]:
def formate2db(df):
     #create_new_index
    df = df.reset_index(drop=True)
    df = create_index(df.fillna(''))
    #tokenize and add index
    df['texte']= df['doc_sents'].apply(lambda x : create_texte(x))
    df['index_token'] = df['doc_sents'].apply(lambda x : create_tokens_index(x))
    
    df['publisher']=''
    df['type']='ecrit'
    table_corpus = df[['index_corpus','publisher', 'journal','type']]
    table_corpus = table_corpus.groupby('index_corpus').agg({
    'publisher': 'first',
    'journal':'first',
     'type': 'first'
    }).reset_index()
    print('table corpus : ok')
    
    df['link']=''
    df['ENR_editeur']=''
    df['ENR_droits']=''
    df['ENR_qualite_acoustique']=''
    df['ENR_lieu']=''
    table_enregistrement = df[['index_enregistrement','index_corpus', 'link','date','Type d\'article','ENR_editeur','categorie','ENR_droits','Taille','ENR_qualite_acoustique', 'ENR_lieu']]#'path_xml',
    table_enregistrement = table_enregistrement.groupby('index_enregistrement').agg({
    'index_corpus': 'first',
    'link': 'first',
    'date': 'first',
    'Type d\'article' : 'first',
    'ENR_editeur' : 'first',
    'categorie':'first',
    'ENR_droits': 'first',
    'Taille': 'first',
    'ENR_qualite_acoustique': 'first',
    'ENR_lieu': 'first'
    }).reset_index()
    print('table table_enregistrement : ok')
    
    df['debut_tour']=''
    df['fin_tour']=''
    table_tour = df[['index_tour', 'index_enregistrement','index_locuteur', 'texte', 'debut_tour', 'fin_tour']]
    table_tour = table_tour.groupby('index_tour').agg({
    'index_enregistrement': 'first',
    'index_locuteur': 'first',
    'texte' : 'first',
    'debut_tour':'first',
    'fin_tour' : 'first'
    }).reset_index()
    print('table table_tour : ok')


    table_locuteurs = df[['index_locuteur']]
    print('table table_loc: ok')
    
    d = {'id_md': ['1', '2', '3', '4'], 'forme': ['ben', 'mais', 'enfin', 'oui']}
    table_md = pd.DataFrame(d)
    print('table table_md: ok')
    
    table_tour_md = df[['index_token', 'index_tour', 'doc_sents']]
    table_tour_md = table_tour_md.explode('index_token')
#     table_tour_md['forme']= table_tour_md.apply(lambda row : extract_forme2(row['doc_sents']), axis=1)
#     table_tour_md['num_tour'] = table_tour_md.apply(lambda row : extract_nb2(row['doc_sents']), axis=1)
#     table_tour_md['id_md'] = table_tour_md['forme'].apply(lambda x : x.replace('ben', '1').replace('mais', '2').replace('enfin', '3').replace('oui', '4'))
#     table_tour_md = table_tour_md.drop('forme', axis=1)

    return table_corpus, table_enregistrement, table_tour, table_locuteurs, table_tour_md, table_md

In [ ]:
ok1

In [ ]:
def formate2db2(df):
     #create_new_index
    df['texte'] = df['text'].fillna('')
    df['doc_sents'] = blabla(nlp, df['text'])
    df = df.explode('doc_sents')
    df['texte']= df['doc_sents'].apply(lambda x : x.text)
    df = df.fillna('')
    df = create_index(df, 'LEMONDEDIPLOMATIQUE')
    df = df.drop('text', axis=1)
    
    #tokenize and add index
    df['MD']=df['doc_sents'].apply(lambda x: get_MD(x))
    
    df['publisher']='ELRA'
    df['type']='écrit'
    df['author_cor']=''
    df['description_cor']='Archives du journal Le Monde Diplomatique'
    table_corpus = df[['index_corpus','publisher', 'author_cor','type', 'description_cor']]
    table_corpus = table_corpus.groupby('index_corpus').agg({
    'publisher': 'first',
    'author_cor':'first',
     'type': 'first', 
    'description_cor':'first'
    }).reset_index()
    print('table corpus : ok')

    
    df['link']=''
    df['ENR_droits']='CC-BY-NC'
    df['ENR_qualite_acoustique']=''
    df['ENR_lieu']=''
    df['categorie']='article de journal'
    df['Taille']=''
    table_enregistrement = df[['index_sub_corpus','index_corpus', 'link','date','mots_clefs','author','categorie','ENR_droits','Taille','ENR_qualite_acoustique', 'ENR_lieu']]#'path_xml',
    table_enregistrement = table_enregistrement.groupby('index_sub_corpus').agg({
    'index_corpus': 'first',
    'link': 'first',
    'date': 'first',
    'mots_clefs' : 'first',
    'author' : 'first',
    'categorie':'first',
    'ENR_droits': 'first',
    'Taille': 'first',
    'ENR_qualite_acoustique': 'first',
    'ENR_lieu': 'first'
    }).reset_index()
    print('table table_enregistrement : ok')
    
#     return df, table_enregistrement, table_corpus
    df['debut_tour']=''
    df['fin_tour']=''
    table_tour = df[['index_turn_index', 'index_sub_corpus', 'texte', 'debut_tour', 'fin_tour']]
    table_tour = table_tour.groupby('index_turn_index').agg({
    'index_sub_corpus': 'first',
    'texte' : 'first',
    'debut_tour':'first',
    'fin_tour' : 'first'
    }).reset_index()
    print('table table_tour : ok')


    table_locuteurs = df[['index_spk','author']]
    table_locuteurs = table_locuteurs.groupby('index_spk').agg({
    'author': 'first'
    }).reset_index()
    print('table table_loc: ok')
    
    table_spk_utt = df[['index_turn_index', 'index_spk']]
    table_spk_utt = table_spk_utt.explode('index_spk')
    print('table spk_utt: ok')
    
    d = {'id_md': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], 'forme': ['car','bref','comme','donc','enfin','ensuite','après','puis','puisque','soudain','mais', 'ben']}
    table_md = pd.DataFrame(d)
    print('table table_md: ok')
    
    table_tour_md = df[['index_turn_index', 'MD']]
    table_tour_md = table_tour_md.explode('MD')
    table_tour_md = table_tour_md.dropna(axis='rows')
    table_tour_md['position'] = table_tour_md['MD'].apply(lambda x : x[0])
    table_tour_md['form'] = table_tour_md['MD'].apply(lambda x : x[1])
    table_tour_md['MD']= table_tour_md['form'].apply(lambda x : map_string_to_integer(x))
#     table_tour_md['forme']= table_tour_md.apply(lambda row : extract_forme2(row['doc_sents']), axis=1)
#     table_tour_md['num_tour'] = table_tour_md.apply(lambda row : extract_nb2(row['doc_sents']), axis=1)
#     table_tour_md['id_md'] = table_tour_md['forme'].apply(lambda x : x.replace('ben', '1').replace('mais', '2').replace('enfin', '3').replace('oui', '4'))
#     table_tour_md = table_tour_md.drop('forme', axis=1)

    return table_corpus, table_enregistrement, table_tour, table_locuteurs, table_tour_md, table_spk_utt, table_md

In [ ]:
table_corpus, table_enregistrement, table_tour, table_locuteurs, table_tour_md, table_spk_utt, table_md = formate2db2(ok)

In [ ]:
table_tour_md

In [202]:
import mysql.connector as msql
from mysql.connector import Error

In [204]:
def pop_corpus(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()

            for i, row in table.iterrows():
                sql =  "INSERT INTO corpus (id_corpus, publisher_corpus, author_corpus, type_corpus, description_corpus) VALUES (%s, %s, %s, %s, %s);"
                cursor.execute(sql, (row['index_corpus'], row['publisher'], row['author_cor'], row['type'], row['description_cor']))
                print("Record inserted into tour")
            conn.commit()
            print("Data insertion completed.")
    except Error as e:
        print("Table Corpus : Error while connecting to MySQL", e) 
        return False

In [205]:
pop_corpus(table_corpus)

Record inserted into tour
Data insertion completed.


In [207]:
def pop_enregistrement(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()           
            for i, row in table.iterrows():
                sql = "INSERT INTO sub_corpus (id_sub_corpus, id_corpus, link_sub_corpus, date_sub_corpus, description_sub_corpus, author_sub_corpus, type_sub_corpus, `right_sub_corpus`, duration_sub_corpus, acoustic_quality_sub_corpus, place_sub_corpus) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
                cursor.execute(sql, (row['index_sub_corpus'], row['index_corpus'], row['link'], row['date'], row['mots_clefs'], row['author'],row['categorie'],row['ENR_droits'],row['Taille'],row['ENR_qualite_acoustique'],row['ENR_lieu']))
                print("Record inserted into tour")
            conn.commit()
            print("Data insertion completed.")
    except Error as e:
        print("Table enregistrement : Error while connecting to MySQL", e) 
        return False

In [208]:
pop_enregistrement(table_enregistrement)

Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Data insertion completed.


In [211]:
def pop_locuteurs(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()                     
            for i, row in table.iterrows():
                sql = "INSERT INTO speaker (id_speaker, name_speaker, age_speaker, gender_speaker, profession_speaker, birth_place_speaker, education_speaker, french_status_speaker, notes_speaker) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
#                 cursor.execute(sql, (row['index_locuteur'], row['LOC_nom'], row['LOC_age'], row['LOC_sexe'], row['LOC_profession'], row['LOC_lieu_naissance'], row['LOC_etude'], row['LOC_statut_langue'], row['notes']))
                cursor.execute(sql, (row['index_spk'], row['author'], '', '', '', '', '', '', ''))
                print("Record inserted into tour")
            conn.commit()
            print("Data insertion completed.")
    except Error as e:
        print("Table locuteurs : Error while connecting to MySQL", e) 
        return False

In [212]:
pop_locuteurs(table_locuteurs)

Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Record inserted into tour
Data insertion completed.


In [214]:
def pop_tour(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()        
            for i, row in table.iterrows():
                sql = "INSERT INTO utterance (id_utterance, id_sub_corpus, text_utterance, start_utterance, end_utterance) VALUES (%s, %s, %s, %s, %s)"
                cursor.execute(sql, (row['index_turn_index'], row['index_sub_corpus'], row['texte'], row['debut_tour'],row['fin_tour']))
                #print("Record inserted into tour")
            conn.commit()
            #print("Data insertion completed.")
    except Error as e:
        print("Table Tour : Error while connecting to MySQL", e) 
        return False

In [215]:
pop_tour(table_tour)

In [218]:
def pop_md(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()
            for i, row in table.iterrows():
                sql = "INSERT INTO dm (id_dm, form_dm) VALUES (%s, %s)"
                cursor.execute(sql, (row['id_md'], row['forme']))          

            # Commit the changes
            conn.commit()
            print("Data insertion completed.")
    except Error as e:
        print("Table md : Error while connecting to MySQL", e)
        return False

In [219]:
pop_md(table_md)

Data insertion completed.


In [216]:
table_tour_md

,index_turn_index,MD,position,form
0,lemondedip_0501117960_7,3,18,comme
0,lemondedip_0501117960_11,2,73,bref
0,lemondedip_0501117960_11,3,88,comme
0,lemondedip_0501117960_45,11,51,mais
0,lemondedip_0501117960_62,11,7,mais
...,...,...,...,...
6,lemondedip_0501118030_42,4,23,donc
8,lemondedip_0501118080_6,7,27,après
8,lemondedip_0501118080_27,1,17,car
8,lemondedip_0501118080_41,4,9,donc


In [226]:
def pop_tour_md(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()
            for i, row in table.iterrows():
                sql = "INSERT INTO dm_utterance (id_dm, id_utterance, position) VALUES (%s, %s, %s)"
                cursor.execute(sql, (row['MD'], row['index_turn_index'], row['position']))          

            # Commit the changes
            conn.commit()
            print("Data insertion completed.")
    except Error as e:
        print("Table tour_md : Error while connecting to MySQL", e)
        return False

In [236]:
table_tour_md

,index_turn_index,MD,position,form
0,lemondedip_0501117960_7,3,18,comme
0,lemondedip_0501117960_11,2,73,bref
0,lemondedip_0501117960_11,3,88,comme
0,lemondedip_0501117960_45,11,51,mais
0,lemondedip_0501117960_62,11,7,mais
...,...,...,...,...
6,lemondedip_0501118030_42,4,23,donc
8,lemondedip_0501118080_6,7,27,après
8,lemondedip_0501118080_27,1,17,car
8,lemondedip_0501118080_41,4,9,donc


In [237]:
pop_tour_md(table_tour_md)

Data insertion completed.


In [240]:
def pop_spk_utt(table):
    try:
        conn = msql.connect(host='localhost',
                            database='codim_db', user='root',
                            password='')
        if conn.is_connected():
            cursor = conn.cursor()
            for i, row in table.iterrows():
                sql = "INSERT INTO speaker_utterance (id_speaker, id_utterance) VALUES (%s, %s)"
                cursor.execute(sql, (row['index_spk'], row['index_turn_index']))          

            # Commit the changes
            conn.commit()
            print("Data insertion completed.")
    except Error as e:
        print("Table tour_md : Error while connecting to MySQL", e)
        return False

In [241]:
pop_spk_utt(table_spk_utt)

Data insertion completed.


In [234]:
list(set(table_tour_md['MD'].tolist()))

['6', '2', '4', '7', '1', 'Not found', '5', '3', '11', '9', '8']

In [235]:
table_tour_md = table_tour_md[table_tour_md.MD != 'Not found']

In [ ]:
# table_tour_md
table_tour_md = table_tour_md.dropna(axis='rows')

In [ ]:
table_tour_md['position'] = table_tour_md['MD'].apply(lambda x : x[0])
table_tour_md['form'] = table_tour_md['MD'].apply(lambda x : x[1])

In [ ]:
def map_string_to_integer(input_string):
    string_to_int_mapping = {
        'car': '1',
        'bref': '2',
        'comme': '3',
        'donc': '4',
        'enfin': '5',
        'ensuite': '6',
        'après': '7',
        'puis': '8',
        'puisque': '9',
        'soudain': '10',
        'mais': '11',
        'ben': '12'
    }
    if isinstance(input_string.text, str):
        # Handle single string input
        return string_to_int_mapping.get(input_string.text, "Not found")
    elif isinstance(input_string, list):
        # Handle list of strings input
        int_list = [string_to_int_mapping.get(string.text, "Not found") for string in input_string]
        return int_list
    else:
        return "Invalid input"

In [ ]:
table_tour_md['MD']= table_tour_md['form'].apply(lambda x : map_string_to_integer(x))

In [ ]:
def extract_nb2(tokens):
    ls = ['ben', 'mais', 'enfin', 'oui']
    list_ind = []
    for ind, x in enumerate(list_):
        if x.text.lower() in ls:
            list_ind.append(ind)
    try : 
        return int(list_ind[0])
    except:
        return None
        
def extract_forme2(tokens):
    ls = ['ben', 'mais', 'enfin', 'oui']
    list_forme = []
    for ind, x in enumerate(list_):
        print(x.text)
        if x.text.lower() in ls:
            list_forme.append(x.text)
    print('----')
    try:
        return str(list_forme[0])
    except:
        return None

In [ ]:
for x in ok1['MD'].tolist():
    if len(x)>0:
        for xx, yy in x:
            print(xx)
            print(yy)

In [ ]:
table_corpus, table_enregistrement, table_tour, table_locuteurs, table_tour_md, table_md = formate2db(ok)

In [ ]:
table_tour_md['forme']= table_tour_md['doc_sents']

In [ ]:
boolean = not table_tour_md['id_md'].is_unique

In [ ]:
boolean

In [ ]:
df = table_tour_md[table_tour_md['id_md'].notna()]

In [ ]:
df = df.dropna(subset=['id_md'])

In [ ]:
ex = []
for x in table_tour_md['forme'].tolist():
    if x: 
        ex.append(x)

In [ ]:
table_tour_md

In [ ]:
for x in table_tour_md1['forme'].tolist():
    if x:
        print(x)

In [ ]:
for x in table_tour_md['id_md'].tolist():
    if x!= '':
        print(x)

In [ ]:
def hihisvp(df):
    #df = pd.read_csv(file, sep='\t', encoding='utf8')
 
    df['tour_md']=df['enonce'].apply(lambda x : tokenize(clean_diacritics(x)))
    df['index_tour_md'] = df.apply(lambda x: create_token_index(x['tour_md'], x['index_tour']), axis=1)
    
    
    table_tour_md = df[['tour_md', 'index_tour_md', 'index_tour']]
    table_tour_md = table_tour_md.explode('index_tour_md')
    table_tour_md = table_tour_md[table_tour_md['index_tour_md'].notna()]
    table_tour_md['forme']= table_tour_md.apply(lambda row : extract_forme2(row['tour_md'], row['index_tour_md']), axis=1)
    table_tour_md['num_tour'] = table_tour_md.apply(lambda row : extract_nb2(row['tour_md'], row['index_tour_md']), axis=1)
    table_tour_md['id_md'] = table_tour_md['forme'].apply(lambda x : x.replace('ben', '1').replace('mais', '2').replace('enfin', '3').replace('oui', '4'))
    table_tour_md = table_tour_md.drop('tour_md', axis=1)
    table_tour_md = table_tour_md.drop('forme', axis=1)
        
    table_tour = df[['index_tour', 'index_enregistrement','index_locuteur', 'enonce', 'debut_tour', 'fin_tour']]
    table_tour = table_tour.groupby('index_tour').agg({
    'index_enregistrement': 'first',
    'index_locuteur': 'first',
    'enonce' : 'first',
    'debut_tour':'first',
    'fin_tour' : 'first'
    }).reset_index()

    
    return table_md#table_corpus, table_enregistrement, table_tour, table_locuteurs, table_tour_md, table_md

In [ ]:
def blabla(nlp, string):
    doc = nlp(string)
    return doc.sents

In [ ]:
hihi = blabla(nlp, str(ok.iloc[:1]['texte'].tolist()))

In [ ]:
for sent in hihi:
    for token in sent:
        print(token.text)
    print("-----") 

In [ ]:
spacy.cli.download("fr_core_news_sm")

In [ ]:
nlp=spacy.load("fr_core_news_sm")
doc=nlp("D'accord pour vendredi 14h. Je ferai un lien Teams.Et pour l'autre rendez-vous concernant les cours, attendons encore un peu pour voir si Nocolas Gony nous répond. Sinon, je vous montrerai quand même. Ce qui sera fait ne sera plus à faire.Bonne journée,Bien à vous")

In [ ]:
for sent in doc.sents:
    print(type(sent))
    for token in sent:
        print(token.text, token.pos_, token.dep_)
    print("-----")